This notebook is for making practice Wfirst image files that can run through sndrizpipe. It will reduce the size of the files to test the capabilities of the workpipe.

In [1]:
from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.wcs import WCS
import os
import numpy as np


def get_list_of_non_flts(directory_name):
    temp_file_list = os.listdir(directory_name)
    files_to_change = []

    for file in temp_file_list:
        if file.endswith('_flt.fits'):
            pass
        else:
            files_to_change.append(file)

    return files_to_change


def get_the_data_out(directory_name, file_name):
    data_temp = fits.open(directory_name + '/' + file_name)
    header_temp  = data_temp[0].header
    image_temp = data_temp[0].data
    data_temp.close()
    '''
    #these next few steps are only for reducing the image size to make it workable with laptops
    wcs = WCS(header_temp)
    image_temp = Cutout2D(image_temp, position=(2044,2044), size=800, wcs=wcs).data
    #header_temp.update(image_temp.wcs.to_header())
    header_temp.set('NAXIS1', 800)
    header_temp.set('NAXIS2', 800)
    header_temp.set('CRPIX1', 400)
    header_temp.set('CRPIX2', 400)
    '''
    
    return header_temp, image_temp


def calc_idcscale(header):
    #IDCSCALE = np.sqrt(new_header['CD1_1']**2+new_header['CD2_2']**2)*3600
    #This is an old fairly bad approximation that we used to use
    
    IDCSCALE = .1 
    #is a good default if you don't want to do the following approximation or if it is giving you problems
    
    #The current calculation comes from these sources:
    #https://www.stsci.edu/itt/review/DrizzlePac/HTML/ch34.html
    #https://www.stsci.edu/itt/DocsStudyGroup/MultiDrizzle/html/ch4.6.3.html
    
    CD1_1 = header['CD1_1']
    CD1_2 = header['CD1_2']
    CD2_1 = header['CD2_1']
    CD2_2 = header['CD2_2']
    
    Pa_v3 = header['ORIENTAT'] - 180 
    #Be careful and check the orientation of your sim images, you might need to change signs or omit the 180
    
    
    print(IDCSCALE)
    return IDCSCALE



def prep_new_header(old_header,filename):
    #All of this header information may need to be changed depending on where the files come from
    #I've just pulled most of these 
    new_header = old_header
    new_header.set('Detector','IR')
    new_header.set('PROPOSID','Rodney_Group')
    new_header.set('LINENUM','Caleb did this')
    new_header.set('TARGNAME','Temp for Wfirst images')
    new_header.set('EXPTIME', 600)
    new_header.set('ROOTNAME', filename.split('.')[0])
    new_header.set('INSTRUME', 'WFC3')
    new_header.set('NGOODPIX', 4088*4088-1)
    new_header.set('EXPNAME', 'Generated Image')
    new_header.set('extnum', 1)
    new_header.set('MEANDARK', 3.20642E-01) #Recommended value from wfc3_dhb handbook
    new_header.set('PA_V3', old_header['PA_FPA'])
    new_header.set('GAIN', 2.5)
    new_header.set('CCDAMP', 'ABCD')
    new_header.set('CCDGAIN', 2.5)
    new_header.set('CCDOFSAB', 190)
    new_header.set('CCDOFSCD', 190)
    new_header.set('ATODGNA', 2.34)
    new_header.set('ATODGNB', 2.37)
    new_header.set('ATODGNC', 2.33)
    new_header.set('ATODGND', 2.36)
    new_header.set('READNSEA', 20.2)
    new_header.set('READNSEB', 19.7)
    new_header.set('READNSEC', 20.1)
    new_header.set('READNSED', 20.6)
    new_header.set('RDNOISE', 1)
    new_header.set('IDCSCALE', calc_idcscale(new_header))
    new_header.set('BUNIT', 'ELECTRONS/S')
    # units for data are in flux rates (counts/s)
    new_header.set('PFLTFILE', 'iref$uc721145i_pfl.fits')
    return new_header


def create_sci_ext(data, new_header):
    new_header.set('EXTVER', 1) #Dr. Rodney recommended this
    return fits.ImageHDU(data=data,header=new_header, name='SCI')


def create_err_ext(data, new_header):
    new_header.set('EXTVER', 1) #ACS follows each chip gets the same number
    return fits.ImageHDU(data=np.sqrt(abs(data)),header=new_header,name='ERR')


def create_dq_ext(data, new_header):
    new_header.set('EXTVER', 1) #ACS follows each chip gets the same number
    dq_array = np.zeros(data.shape,dtype='int16')
    problem_values = np.argwhere(~np.isfinite(data))
    
    for index in problem_values:
        dq_array[index[0]][index[1]]+=1
    return fits.ImageHDU(data=dq_array, header=new_header,name='DQ')

def create_time_ext(data, new_header):
    new_header.set('EXTVER', 1)
    new_header.set('BUNIT', 'SECONDS')
    #I am using 600 seconds because that was a good round number
    return fits.ImageHDU(data=np.ones(data.shape,dtype='float32')*600, header=new_header,name='TIME')


def make_new_fits_file(path,from_directory, to_directory, filename):
    os.chdir(path)
    old_header, data= get_the_data_out(path + '/' + from_directory,filename)
    new_header = prep_new_header(old_header,filename)
    fit0 = fits.PrimaryHDU(header=new_header)
    fit0.header.set('EXTNAME', 'Primary')
    fit0.header.set('EXTVER',1)
    fit1 = create_sci_ext(data, new_header)
    fit2 = create_err_ext(data, new_header)
    fit3 = create_dq_ext(data, new_header)
    fit4 = create_time_ext(data, new_header)

    new_fits_file = fits.HDUList([fit0,fit1,fit2,fit3,fit4])
    main_name = filename.split('.')[0]
    os.chdir(to_directory)
    new_fits_file.info()
    new_fits_file.writeto(main_name + '_flt.fits',overwrite=True)

    
def convert_Wfirstsim_to_flt(directory_name):
    new_directory = directory_name +'.flt'
    os.mkdir(new_directory)
    
    files_to_change = get_list_of_non_flts(directory_name)
    
    path = os.getcwd()
    
    for file in files_to_change:
        make_new_fits_file(path,directory_name,new_directory,file)

In [2]:
convert_Wfirstsim_to_flt('rotate')

-9.001531504503415e-10
Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  Primary       1 PrimaryHDU      90   ()      
  1  SCI           1 ImageHDU        93   (4088, 4088)   float32   
  2  ERR           1 ImageHDU        93   (4088, 4088)   float32   
  3  DQ            1 ImageHDU        93   (4088, 4088)   int16   
  4  TIME          1 ImageHDU        93   (4088, 4088)   float32   
-9.228749148977118e-10
Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  Primary       1 PrimaryHDU      90   ()      
  1  SCI           1 ImageHDU        93   (4088, 4088)   float32   
  2  ERR           1 ImageHDU        93   (4088, 4088)   float32   
  3  DQ            1 ImageHDU        93   (4088, 4088)   int16   
  4  TIME          1 ImageHDU        93   (4088, 4088)   float32   
-9.00153150450342e-10
Filename: (No file associated with this HDUList)
No.    Name      Ve

In [1]:
from astropy.io import fits

fix_file = 'rotate.refim/rotate_wcsref_drz_sci.fits'
fix_data = fits.open(fix_file,mode='update')
fix_data[0].header.set('IDCSCALE', .1)
fix_data.flush()
fix_data.close()